## Project


### Importaciones


In [20]:
import pandas as pd

Lectura del data set (se necesito separar por ; para su correcta lectura)

In [ ]:
covid = pd.read_csv("Datos interesantes/Casos_positivos_de_COVID-19_en_Colombia._20250112.csv", sep=";")
covid.head()

C:\Users\aleja\AppData\Local\Temp\ipykernel_16724\3041061035.py:1: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  covid = pd.read_csv("Datos interesantes/Casos_positivos_de_COVID-19_en_Colombia._20250112.csv", sep=";", nrows=500000)


,fecha reporte web,ID de caso,Fecha de notificación,Código DIVIPOLA departamento,Nombre departamento,Código DIVIPOLA municipio,Nombre municipio,Edad,Unidad de medida de edad,Sexo,...,Código ISO del país,Nombre del país,Recuperado,Fecha de inicio de síntomas,Fecha de muerte,Fecha de diagnóstico,Fecha de recuperación,Tipo de recuperación,Pertenencia étnica,Nombre del grupo étnico
0,2020-12-24 00:00:00,"1,556,979",2020-12-22 00:00:00,76,VALLE,"76,001",CALI,67,1,F,...,NaN,NaN,Recuperado,2020-12-21 00:00:00,NaN,2020-12-23 00:00:00,2021-01-04 00:00:00,Tiempo,6,NaN
1,2020-12-24 00:00:00,"1,556,980",2020-12-19 00:00:00,76,VALLE,"76,001",CALI,66,1,F,...,NaN,NaN,Recuperado,2020-12-07 00:00:00,NaN,2020-12-23 00:00:00,2020-12-25 00:00:00,Tiempo,6,NaN
2,2020-12-24 00:00:00,"1,556,981",2020-12-19 00:00:00,76,VALLE,"76,001",CALI,68,1,F,...,NaN,NaN,Recuperado,2020-12-18 00:00:00,NaN,2020-12-22 00:00:00,2021-01-01 00:00:00,Tiempo,6,NaN
3,2020-12-24 00:00:00,"1,556,982",2020-12-22 00:00:00,76,VALLE,"76,001",CALI,74,1,F,...,NaN,NaN,Fallecido,2020-12-17 00:00:00,2020-12-30 00:00:00,2020-12-23 00:00:00,NaN,NaN,6,NaN
4,2020-12-24 00:00:00,"1,556,983",2020-12-22 00:00:00,76,VALLE,"76,001",CALI,65,1,F,...,NaN,NaN,Recuperado,2020-12-21 00:00:00,NaN,2020-12-23 00:00:00,2021-01-04 00:00:00,Tiempo,6,NaN


Información de las variables

In [23]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   fecha reporte web             500000 non-null  object
 1   ID de caso                    500000 non-null  object
 2   Fecha de notificación         500000 non-null  object
 3   Código DIVIPOLA departamento  500000 non-null  object
 4   Nombre departamento           500000 non-null  object
 5   Código DIVIPOLA municipio     500000 non-null  object
 6   Nombre municipio              500000 non-null  object
 7   Edad                          500000 non-null  int64 
 8   Unidad de medida de edad      500000 non-null  int64 
 9   Sexo                          500000 non-null  object
 10  Tipo de contagio              500000 non-null  object
 11  Ubicación del caso            497693 non-null  object
 12  Estado                        497693 non-null  object
 13 

### **Inicio proceso de limpieza**
Eliminación de duplicados

In [26]:
duplicados = covid.duplicated()
covid = covid.drop_duplicates()
covid.isna().sum()

fecha reporte web                    0
ID de caso                           0
Fecha de notificación                0
Código DIVIPOLA departamento         0
Nombre departamento                  0
Código DIVIPOLA municipio            0
Nombre municipio                     0
Edad                                 0
Unidad de medida de edad             0
Sexo                                 0
Tipo de contagio                     0
Ubicación del caso                2307
Estado                            2307
Código ISO del país             499763
Nombre del país                 499763
Recuperado                        1871
Fecha de inicio de síntomas      46315
Fecha de muerte                 484064
Fecha de diagnóstico               271
Fecha de recuperación            15500
Tipo de recuperación             15500
Pertenencia étnica                   0
Nombre del grupo étnico         492620
dtype: int64

Transformación del tipo de variables de fecha a datetime

In [27]:
covid['Fecha de inicio de síntomas'] = pd.to_datetime(covid['Fecha de inicio de síntomas'], errors='coerce')
covid['Fecha de diagnóstico'] = pd.to_datetime(covid['Fecha de diagnóstico'], errors='coerce')
covid['Fecha de muerte'] = pd.to_datetime(covid['Fecha de muerte'], errors='coerce')
covid['Fecha de recuperación'] = pd.to_datetime(covid['Fecha de recuperación'], errors='coerce')

Creación de nuevas variables para contestar las preguntas de negocio

In [ ]:
covid['sobrevivio'] = covid['Fecha de muerte'].isna().astype(int)
covid['dias_diagnostico'] = (covid['Fecha de diagnóstico'] - covid['Fecha de inicio de síntomas']).dt.days
analisis = covid[['dias_diagnostico', 'sobrevivio']].dropna()

In [31]:
columns_with_missing = covid.isna().sum()[covid.isna().sum() > 0]
columns_with_missing.sort_values(ascending=False)

Código ISO del país            499763
Nombre del país                499763
Nombre del grupo étnico        492620
Fecha de muerte                484064
dias_diagnostico                46565
Fecha de inicio de síntomas     46315
Fecha de recuperación           15500
Tipo de recuperación            15500
Ubicación del caso               2307
Estado                           2307
Recuperado                       1871
Fecha de diagnóstico              271
dtype: int64

In [32]:
threshold_colum = len(covid) * 0.3
cols_to_drop = covid.columns[covid.isna().sum() > threshold_colum]
cols_to_drop = cols_to_drop.append(pd.Index(['ID de caso']))
print(cols_to_drop)

Index(['Código ISO del país', 'Nombre del país', 'Fecha de muerte',
       'Nombre del grupo étnico', 'ID de caso'],
      dtype='object')


In [33]:
covid.drop(columns=cols_to_drop, inplace=True)

In [34]:
missing_values_after_drop = covid.isnull().sum()
columns_with_missing_after_drop = missing_values_after_drop[missing_values_after_drop > 0]
columns_with_missing_after_drop 

Ubicación del caso              2307
Estado                          2307
Recuperado                      1871
Fecha de inicio de síntomas    46315
Fecha de diagnóstico             271
Fecha de recuperación          15500
Tipo de recuperación           15500
dias_diagnostico               46565
dtype: int64

In [35]:
categorical_columns = ['Ubicación del caso', 'Estado', 'Recuperado', 'Fecha de recuperación', 'Tipo de recuperación']
for col in categorical_columns:
    covid[col] = covid[col].fillna(covid[col].mode()[0])

Eliminar variables redundantes


In [36]:
covid = covid.drop(columns=["Código DIVIPOLA departamento","Código DIVIPOLA municipio"])

In [37]:
covid.head()

,fecha reporte web,Fecha de notificación,Nombre departamento,Nombre municipio,Edad,Unidad de medida de edad,Sexo,Tipo de contagio,Ubicación del caso,Estado,Recuperado,Fecha de inicio de síntomas,Fecha de diagnóstico,Fecha de recuperación,Tipo de recuperación,Pertenencia étnica,sobrevivio,dias_diagnostico
0,2020-12-24 00:00:00,2020-12-22 00:00:00,VALLE,CALI,67,1,F,Comunitaria,Casa,Leve,Recuperado,2020-12-21,2020-12-23,2021-01-04,Tiempo,6,1,2.0
1,2020-12-24 00:00:00,2020-12-19 00:00:00,VALLE,CALI,66,1,F,Comunitaria,Casa,Leve,Recuperado,2020-12-07,2020-12-23,2020-12-25,Tiempo,6,1,16.0
2,2020-12-24 00:00:00,2020-12-19 00:00:00,VALLE,CALI,68,1,F,Comunitaria,Casa,Leve,Recuperado,2020-12-18,2020-12-22,2021-01-01,Tiempo,6,1,4.0
3,2020-12-24 00:00:00,2020-12-22 00:00:00,VALLE,CALI,74,1,F,Comunitaria,Fallecido,Fallecido,Fallecido,2020-12-17,2020-12-23,2021-06-01,Tiempo,6,0,6.0
4,2020-12-24 00:00:00,2020-12-22 00:00:00,VALLE,CALI,65,1,F,Comunitaria,Casa,Leve,Recuperado,2020-12-21,2020-12-23,2021-01-04,Tiempo,6,1,2.0
